In [1]:
import os
import sys

In [3]:
import zipfile

destination_folder = 'unzipped_files/'
zf = zipfile.ZipFile('Mgt_&_OB_1980_2015.zip', 'r')
zf

<zipfile.ZipFile filename='Mgt_&_OB_1980_2015.zip' mode='r'>

In [5]:
#TODO: write function to clear destination_folder
for file in zf.namelist():
    if file.startswith('metadata/journal'):
        zf.extract(file, destination_folder)

In [4]:
path, dirs, files = next(os.walk(destination_folder+ 'metadata/'))
file_count = len(files)
file_count

25000

In [45]:
import pandas as pd
import xml.etree.ElementTree as ET

folder = destination_folder + 'metadata/'

def add(elem, attrs):
    elements = attrs['elements']
    surnames = attrs['surname']
    given_names = attrs['given-names']
    tag = elem.tag
    if tag in attrs['df_cols']:
        if tag == 'journal-id':
            tag = 'journal_id'
        elif tag == 'article':
            article_attrs = elem.attrib
            article_type = article_attrs['article-type']
            tag = 'type'
            elem.text = article_type
        elif tag =='journal-title':
            tag = 'journal_title'
        elif tag == 'article-id':
            tag = 'article_id'
        elif tag == 'article-name':
            tag = 'article_name'
        elif tag == 'surname':
            surnames.append(elem.text.split(',')[0])
            elem.text = surnames
        elif tag == 'given-names':
            tag = 'given_names'
            given_names.append(elem.text)
            elem.text = given_names
        elif tag == 'issue-id':
            tag = 'issue_id'
        elif tag == 'ns1:ref':
            tag = 'jstor_url'
        elif tag == 'p':
            tag = 'abstract'
            
#        if tag not in elements:
#            elements[tag] = pd.Series([elem.text])
#        else:
#            elements[tag].append(pd.Series([elem.text]), ignore_index=True)
        len_list = len(elements[tag])
        elements[tag][len_list - 1] = elem.text
        
    for child in elem.findall('*'):
        add(child, attrs)

def xml2df():
    all_records = {'type':[], 'journal_id':[], 'journal_title':[], 'issn':[], 'article_id':[], 'article_name':[], 'given_names':[], 'surname':[], 'day':[], 'month':[], 'year':[], 'volume':[], 'issue':[], 'issue_id':[], 'fpage':[], 'lpage':[], 'jstor_url':[], 'abstract':[]}
    attrs = {}
    attrs['elements'] = all_records
    attrs['surname'] = []
    attrs['given-names'] = []
    attrs['df_cols'] = ['article', 'journal-id', 'journal-title', 'issn', 'article-id', 'article-name', 'given-names', 'surname', 'day', 'month', 'year', 'volume', 'issue', 'issue-id', 'fpage', 'lpage', 'ns1:ref', 'p']

    files = [folder + 'journal-article-10.2307_975518.xml', folder + 'journal-article-10.2307_9752.xml']
    for file in files:
        t = ET.parse(file) # element tree
        root = t.getroot()
        for record in all_records:
            all_records[record].append('None')
        add(root, attrs)
        
    #print(all_records)
    df = pd.DataFrame(all_records)
    return df

df = xml2df()
df

FileNotFoundError: [Errno 2] No such file or directory: 'unzipped_files/metadata/journal-article-10.2307_97525.xml'